In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [12]:
# Chrome WebDriver 초기화
driver = webdriver.Chrome()

In [14]:
# 스크랩할 웹사이트의 URL
url = 'https://teacher.i-scream.co.kr/help/afterword/list.do?sso=ok'

In [15]:
# 웹페이지 열기
driver.get(url)

In [16]:
# 첫 번째 수업 이름 요소를 XPath로 찾기
driver.find_element(By.XPATH,'//*[@id="content_list"]/div[3]/ul/li[1]/div[1]/div[1]/a')

<selenium.webdriver.remote.webelement.WebElement (session="6d63b1a07b6a7b7e67d8542291c59735", element="f.FDAC0EEE0AA4141BE3770B2864D22C2F.d.61DB435DC38891D6C9D94B5168FB4963.e.50")>

In [17]:
# 첫 번째 수업 이름 요소에서 텍스트(수업 이름) 추출해서 확인
driver.find_element(By.XPATH,'//*[@id="content_list"]/div[3]/ul/li[1]/div[1]/div[1]/a').text

'그림으로 배우는 한국사 이미지 노트 2'

In [18]:
# 두 번째 수업 이름 요소에서 텍스트(수업 이름) 추출해서 확인
driver.find_element(By.XPATH,'//*[@id="content_list"]/div[3]/ul/li[2]/div[1]/div[1]/a').text

'흔들리지 않는 교실을 위한 단호한 학급운영'

In [13]:
# 수업 이름을 저장할 빈 리스트 초기화
class_name = []

# 첫 번째부터 열 번째까지의 수업 이름을 추출하는 반복문(한 페이지에 10개)
for class_num in range(1, 11):  
    x = driver.find_element(
        By.XPATH,
        f'//*[@id="content_list"]/div[3]/ul/li[{class_num}]/div[1]/div[1]/a'
    ).text
    class_name.append(x)

In [15]:
#추출한 모든 수업 이름 확
class_name

['알베르토가 함께 하는 핵심역량 연계 다문화 교육',
 'MBTI로 가화만사성! 성격유형으로 보는 부부&자녀 대화법',
 '학습격차를 줄이기 위한 슬기로운 수업방안',
 '부자 교사의 재테크',
 '수업이 막막할 땐 성공 보장 수업 스킬',
 '교사를 위한 학생안전길라잡이',
 '특수교사 백과사전',
 '[700+목표] 한 권으로 끝내는 해커스 토익 700+RC ①',
 '즐거운 대화, 하브루타로 수업하기',
 '큰별쌤 최태성의 별★별한국사']

In [34]:
# XPath를 사용하여 리뷰 요소들을 모두 찾음
reviews = driver.find_elements(By.XPATH, '//*[@id="content_list"]/div[3]/ul/li/div[1]/div[2]/div')

# 각 리뷰 요소에 대해 반복
for review in reviews:
    # 별점 요소들의 개수를 세어 별점 평가를 계산
    stars = review.find_elements(By.CLASS_NAME, 'star.checked')
    # 별점 요소들의 개수를 세어 별점 평가를 계산
    rating = len(stars)
    print(rating)

3
5
5
5
5
5
5
5
5
5


In [42]:
# 별점 평가를 저장할 빈 리스트 초기화
rating = []

# 첫 번째부터 열 번째까지의 수업에 대해 반복
for c in range(1, 11):  
    # 현재 수업의 별점 평가를 저장할 빈 리스트 초기화
    current_rating = []
     # XPath를 사용하여 현재 수업의 리뷰 요소들을 모두 찾음
    reviews = driver.find_elements(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{c}]/div[1]/div[2]/div')
    for review in reviews:
        stars = review.find_elements(By.CLASS_NAME, 'star.checked')
         # XPath를 사용하여 현재 수업의 리뷰 요소들을 모두 찾음
        current_rating.append(len(stars))
    # 모든 수업의 별점 평가를 담는 rating 리스트에 current_rating 리스트를 확장하여 추가
    rating.extend(current_rating)

In [44]:
# 모든 수업의 별점 평가 리스트 출력
rating

[3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [9]:
# XPath를 사용하여 차의 요소를 찾고 텍스트를 추출
driver.find_element(By.XPATH,'//*[@id="content_list"]/div[3]/ul/li[2]/div[1]/div[1]').text

'[15차시/학습지도] 학습부진, 천천히 함께 걷기'

In [27]:
# 카테고리를 저장할 빈 리스트 초기화
category = []

# 첫 번째부터 열 번째까지의 항목에 대해 반복
for c in range(1, 11):
    xpath = f'//*[@id="content_list"]/div[3]/ul/li[{c}]/div[1]/div[1]'
    # XPath를 사용하여 현재 항목의 요소 찾기
    element = driver.find_element(By.XPATH, xpath)
    # 요소에서 텍스트 추출
    text = element.text

    # 대괄호 안의 텍스트를 추출하기 위한 정규 표현식 패턴 설정
    pattern = r'\[(.*?)\]'
    # 정규 표현식을 사용하여 패턴과 일치하는 텍스트 찾기
    match = re.search(pattern, text)
    if match:
        extracted_text = match.group(1)  # 그룹 1을 사용하여 대괄호를 제외한 텍스트만 추출
        category.append(extracted_text)  # 추출한 텍스트를 카테고리 리스트에 추가


In [26]:
category

['15차시/학습지도',
 '15차시/학습지도',
 '15차시/학습지도',
 '30차시/생활지도',
 '5차시/자기계발',
 '15차시/학습지도',
 '15차시/자기계발',
 '30차시/학습지도',
 '15차시/생활지도',
 '30차시/학급경영']

# 강의명, 별점, 카테고리 크롤링 

In [27]:
#크롤링 전체 정리 코드

import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹사이트 열기
url = 'https://teacher.i-scream.co.kr/help/afterword/list.do?sso=ok'
driver.get(url)

# 리스트 초기화
class_name = []
rating = []
category = []
chasi = []

# 1페이지부터 2442페이지까지 반복하여 데이터 수집
for page in range(1, 2442):
    print(f"Processing page: {page}")  # 해당 페이지 수집하면 멘트 출력

    # 1부터 10까지의 항목에 대해 class_name, rating, category 데이터 수집
    for c in range(1, 11):  
        # 수업 이름 수집
        try:
            x = driver.find_element(
                By.XPATH,
                f'//*[@id="content_list"]/div[3]/ul/li[{c}]/div[1]/div[1]/a'
            ).text
            class_name.append(x)
        except:
            class_name.append(None) # 요소를 찾지 못하면 None을 추가

        # 별점 수집
        current_rating = []
        reviews = driver.find_elements(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{c}]/div[1]/div[2]/div')
        for review in reviews:
            stars = review.find_elements(By.CLASS_NAME, 'star.checked')
            current_rating.append(len(stars))
        rating.extend(current_rating)

        # 카테고리 수집
        xpath = f'//*[@id="content_list"]/div[3]/ul/li[{c}]/div[1]/div[1]'
        try:
            element = driver.find_element(By.XPATH, xpath)
            text = element.text
            pattern = r'\[(.*?)\]'
            match = re.search(pattern, text)
            if match:
                extracted_text = match.group(1)  # 그룹 1을 사용하여 대괄호를 제외한 텍스트만 추출
                # "15차시/" 부분을 제거하고 뒷부분만 추출
                category_text = re.sub(r'^\d+차시/', '', extracted_text)
                category.append(category_text)
                # 숫자+차시/에서 /를 제거하고 숫자만 추출하여 차시 컬럼에 추가
                chasi_text = re.sub(r'[^0-9]', '', match.group(1))
                chasi.append(chasi_text)
            else: # 패턴 매칭이 실패하면 카테고리와 차시에 None을 추가
                category.append(None)
                chasi.append(None)
        except: # 요소를 찾지 못하면 카테고리와 차시에 None을 추가
            category.append(None)
            chasi.append(None)

    # 다음 페이지로 이동
    next_page = page + 1
    driver.get(f'https://teacher.i-scream.co.kr/help/afterword/list.do?searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&pageIndex={next_page}&sso=ok')

# 데이터 프레임 생성
df = pd.DataFrame({
    'Class Name': class_name,
    'Rating': rating,
    'Category': category,
    '차시': chasi
})

# 데이터 프레임 출력
print(df)

# 데이터프레임을 엑셀 파일로 저장
excel_file = 'iscream_class_rating.xlsx'
df.to_excel(excel_file, index=False)
print(f'Excel file "{excel_file}" created successfully.')


Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12
Processing page: 13
Processing page: 14
Processing page: 15
Processing page: 16
Processing page: 17
Processing page: 18
Processing page: 19
Processing page: 20
Processing page: 21
Processing page: 22
Processing page: 23
Processing page: 24
Processing page: 25
Processing page: 26
Processing page: 27
Processing page: 28
Processing page: 29
Processing page: 30
Processing page: 31
Processing page: 32
Processing page: 33
Processing page: 34
Processing page: 35
Processing page: 36
Processing page: 37
Processing page: 38
Processing page: 39
Processing page: 40
Processing page: 41
Processing page: 42
Processing page: 43
Processing page: 44
Processing page: 45
Processing page: 46
Processing page: 47
Processing page: 48
Processing page: 49
Processing page: 50
Processin

In [29]:
excel_file = 'iscream_class_rating.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(excel_file)

# 차시 컬럼 값들을 문자열에서 숫자로 변환
df['차시'] = df['차시'].apply(lambda x: int(x) if pd.notnull(x) else None)

# 원본 엑셀 파일에 덮어쓰기
df.to_excel(excel_file, index=False)
print(f'Converted Excel file "{excel_file}" successfully.')

Converted Excel file "iscream_class_rating.xlsx" successfully.


In [21]:
excel_file = 'iscream_class_rating.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(excel_file)


In [22]:
df

,Class Name,Rating,Category,차시
0,직업탐구 - 별일입니다,5,학습지도,30
1,"생각을 꺼내는 교육, 이제는 IB 시대",3,학습지도,15
2,"미래인재의 역량을 키운다, 미래교육 20분",5,자기계발,30
3,KOICA와 함께하는 세계시민교육,5,교과지도,30
4,정재승의 인간수업,5,자기계발,30
...,...,...,...,...
24405,신영일과 함께하는 안전교육 365,5,생활지도,15
24406,정재승의 미래학교,5,학습지도,30
24407,법정의무교육 한 번에 이수하기,5,생활지도,15
24408,하루 20분! 나만을 위한 '나마스떼' 힐링 요가,5,자기계발,15


## 강의 현재 가격 크롤링

In [36]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹사이트 열기
url = 'https://teacher.i-scream.co.kr/course/crs/creditList.do?searchOrdinalTyCode=TY01&searchOrderField=NEW&sso=ok'
driver.get(url)

# 데이터 저장 리스트
data = {
    "강의명": [],
    "가격": []
}

# 강의 크롤링 함수
def crawl_lecture(start_index, end_index, more_clicks):
    # 지정된 시작 인덱스부터 종료 인덱스까지 반복
    for i in range(start_index, end_index + 1):
        try:
            # 더보기 버튼 클릭
            for _ in range(more_clicks):
                more_button_xpath = '//*[@id="divMore"]/span' # 더보기 버튼의 XPath
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, more_button_xpath))) # 더보기 버튼이 클릭 가능할 때까지 대기
                driver.find_element(By.XPATH, more_button_xpath).click() # 더보기 버튼 클릭
                time.sleep(2)  # 페이지 로드 시간 보장

            # 강의 선택
            lecture_xpath = f'//*[@id="content"]/ul/li[{i}]/a' # 강의의 XPath
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, lecture_xpath))) # 강의가 클릭 가능할 때까지 대기
            lecture = driver.find_element(By.XPATH, lecture_xpath) # 강의 요소 찾기
            lecture.click() # 강의 클릭

            # 경고창 확인
            try:
                WebDriverWait(driver, 3).until(EC.alert_is_present()) # 경고창이 나타날 때까지 대기
                alert = driver.switch_to.alert # 경고창으로 전환
                alert.accept() # 경고창 확인 클릭
            except:
                pass   # 경고창이 없으면 넘어감

            # 강의명 크롤링
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="bbs_wrap"]/div[1]/h3/span'))) # 강의명이 나타날 때까지 대기
            lecture_name = driver.find_element(By.XPATH, '//*[@id="bbs_wrap"]/div[1]/h3/span').text  # 강의명 요소 찾기 및 텍스트 추출
            data["강의명"].append(lecture_name)   # 강의명을 리스트에 추가

            # 가격 크롤링
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="crsPrice"]/span')))   # 가격이 나타날 때까지 대기
            price = driver.find_element(By.XPATH, '//*[@id="crsPrice"]/span').text  # 가격 요소 찾기 및 텍스트 추출
            data["가격"].append(price) # 가격을 리스트에 추가

            # 뒤로가기
            driver.back()  # 이전 페이지로 돌아가기
            time.sleep(2)  # 페이지 로드 시간 보장
        except Exception as e:
            print(f"Error crawling lecture {i}: {e}")        # 예외 발생 시 에러 메시지 출력

# 1부터 30번째 강의까지 크롤링 (더보기 클릭 없음)
crawl_lecture(1, 30, 0)

# 31부터 359번째 강의까지 크롤링 (30개 단위로 더보기 클릭 수 증가)
for group_start in range(31, 360, 30):
    group_end = min(group_start + 29, 359)
    more_clicks = (group_start - 1) // 30
    crawl_lecture(group_start, group_end, more_clicks)

# 데이터프레임 생성
df = pd.DataFrame(data)

# 크롤링 결과 출력
print(df)

# 데이터프레임을 엑셀 파일로 저장
excel_file = 'I-scream_prices.xlsx'
df.to_excel(excel_file, index=False)
print(f'Excel file "{excel_file}" created successfully.')

# 드라이버 종료
driver.quit()


                                 강의명       가격
0    대한민국 핫플! 다시 발견하다. 국내 여행 TOP 120  85,000원
1              야나두 스르르 학습지 - 기초 영어 편  85,000원
2            맘껏 먹고 가볍게 살기 with 채소과일식  50,000원
3            랜선 음악 여행, 이탈리아에 숨겨진 클래식  50,000원
4                 교사를 위한 구글 활용법(실전편)  50,000원
..                               ...      ...
354            조승연 작가와 함께하는 서양예술사 산책  65,000원
355                   교실 속 미술관(서양화가)  35,000원
356                   천천히 깊게 읽는 슬로리딩  65,000원
357                  교사를 위한 학생안전길라잡이  35,000원
358              왕초보도 할 수 있는 실전 여행영어  65,000원

[359 rows x 2 columns]
Excel file "lecture_prices.xlsx" created successfully.
